In [ ]:
import sys, os, platform, psutil, sys, tempfile
import scvi, sklearn, scipy, anndata, scanpy as sc, seaborn as sns, torch, numpy as np, muon

print("Python:", sys.version.split()[0])
print("OS:", platform.system(), platform.release())
print("CPU count:", psutil.cpu_count(logical=True))
print("Memory (GB):", round(psutil.virtual_memory().total / (1024**3), 2))
print ("")
print("scvi-tools:", scvi.__version__)
print("scikit-learn:", sklearn.__version__)
print("scipy:", scipy.__version__)
print("anndata:", anndata.__version__)
print("scanpy:", sc.__version__)
print("seaborn:", sns.__version__)
print("torch:", torch.__version__)
print("numpy:", np.__version__)
print("muon:", muon.__version__)

In [ ]:
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

# Define the output directory relative to the current working directory
output_dir = 'Intermediate Files/Parameter_Testing_07242025'
os.makedirs(output_dir, exist_ok=True)

# You can now use output_dir to save your file s
print(f"Output directory is set to: {output_dir}")

In [ ]:
from anndata import read_h5ad
Int_folder = "Intermediate_Files/QC_07232025/"
# Define the folder path
iso_data_std = read_h5ad(os.path.join(Int_folder, "Concatenated_Iso_Data.h5ad"))

In [ ]:
# Check for initial NaN values
print("Initial NaN values in the PBMC Isoform data matrix:", np.isnan(iso_data_std.X).sum())

In [ ]:
print(iso_data_std.obs["batch"].value_counts())

In [ ]:
iso_data_std.X.dtype

In [ ]:
iso_data_std.layers["counts"] =iso_data_std.X.copy()

In [ ]:
## create a new MuData object where the normalized data are another “modality”
mdata_i_std = muon.MuData({"rna":iso_data_std.copy()}, axis=-1)
## Now rna is count-based and log_norm_rna is log-normalized
mdata_i_std.mod["rna"].X = mdata_i_std.mod["rna"].layers["counts"]
del mdata_i_std.mod["rna"].raw
del mdata_i_std.mod["rna"].layers["counts"]

In [ ]:
mdata_i_std

In [ ]:
## Extract the RNA modality
rna_adata_i_std = mdata_i_std.mod['rna']

## Ensure that the counts layer is set
rna_adata_i_std.layers["counts"] = rna_adata_i_std.X.copy()

In [ ]:
print(rna_adata_i_std.obs["batch"].value_counts())

In [ ]:
rna_adata_i_std.X.dtype

In [ ]:
print(mdata_i_std.mod['rna'].obs.columns)  # List all columns in .obs

In [ ]:
# Check the column's dtype
print(mdata_i_std.mod['rna'].obs['batch'].dtype)

# Display unique values
print(mdata_i_std.mod['rna'].obs['batch'].unique())

In [ ]:
if "_scvi_batch" in mdata_i_std.mod['rna'].obs.columns:
    del mdata_i_std.mod['rna'].obs["_scvi_batch"]

In [ ]:
from scvi.model import AUTOZI

AUTOZI.setup_anndata(
    adata=mdata_i_std.mod['rna'],
    batch_key="batch"  # Specify the batch column
)

In [ ]:
print(mdata_i_std.mod['rna'].obs.columns)  # Should include "_scvi_batch"
print(mdata_i_std.mod['rna'].obs["_scvi_batch"].unique())  # Check the values

In [ ]:
from scvi.data import AnnDataManager
from scvi.data.fields import CategoricalObsField

# Initialize AnnDataManager
adata_manager = AnnDataManager(
    fields=[
        CategoricalObsField("batch", attr_key="_scvi_batch")  # Specify the batch key
    ]
)

# Register fields for the AnnData object
adata_manager.register_fields(mdata_i_std.mod['rna'])

# Validate fields
adata_manager.validate()

# Check that `_scvi_batch` was added
print(mdata_i_std.mod['rna'].obs['_scvi_batch'].unique())

In [ ]:
# Confirm batch column exists and inspect
print(mdata_i_std.mod['rna'].obs["batch"].value_counts())
print(mdata_i_std.mod['rna'].obs["batch"].dtype)

In [ ]:
print(mdata_i_std.mod['rna'].uns.keys())
print(type(mdata_i_std.mod['rna']))
print(mdata_i_std.mod['rna'].obs.columns)

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "127"
os.environ["MKL_NUM_THREADS"] = "127"

In [ ]:
def run_autozi_test(
    adata_full,
    subset_n=1000,
    n_latent=20,
    dropout_rate=0.3,
    learning_rate=1e-2,
    weight_decay=1e-3,
    max_epochs=20,
    seed=42
):
    # === Subset the data ===
    np.random.seed(seed)
    subset_indices = np.random.choice(adata_full.obs_names, size=min(subset_n, adata_full.n_obs), replace=False)
    adata_subset = adata_full[subset_indices].copy()

    # Setup for scVI
    AUTOZI.setup_anndata(adata_subset, batch_key="batch")

    # Initialize model
    model = AUTOZI(
        adata=adata_subset,
        n_latent=n_latent,
        dropout_rate=dropout_rate
    )

    # Train
    model.train(
        max_epochs=max_epochs,
        train_size=0.9,
        early_stopping=True,
        early_stopping_patience=10,
        batch_size=128,
        plan_kwargs={
            "lr": learning_rate,
            "weight_decay": weight_decay
        }
    )

    # Report ELBO
    train_elbo = model.history['elbo_train'].iloc[-1, 0]
    val_elbo = model.history['elbo_validation'].iloc[-1, 0]
    print(f"✅ Finished: latent={n_latent}, dropout={dropout_rate}, lr={learning_rate}, wd={weight_decay}")
    print(f"   Train ELBO: {train_elbo:.2f}, Val ELBO: {val_elbo:.2f}")
    return model

In [ ]:
# === Load your MuData ===
# (Assumes you already loaded it like this)
adata = mdata_i_std.mod['rna']

In [ ]:
# === Run test ===
test_model = run_autozi_test(
    adata_full=adata,
    subset_n=5000,
    n_latent=30,
    dropout_rate=0.4,
    learning_rate=5e-3,
    weight_decay=1e-2
)

In [ ]:
# Access training history
import matplotlib.pyplot as plt

train_elbo = test_model.history['elbo_train']  # Dataset 1
val_elbo = test_model.history['elbo_validation']  # Dataset 1

# Print last few values
print("Training ELBO gene-level:", train_elbo[-10:])
print("Validation ELBO gene-level:", val_elbo[-10:])

In [ ]:
plt.plot(train_elbo, label="Training ELBO")
plt.plot(val_elbo, label="Validation ELBO")
plt.xlabel("Epoch")
plt.ylabel("Negative ELBO")
plt.legend()
plt.title("Training vs Validation Loss (ELBO) Gene-Level")
plt.show()

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)

from sklearn.metrics import silhouette_score
print("silhouetts mixing score:", silhouette_score(latent, test_model.adata.obs['batch']))
sc.pl.umap(test_model.adata, color="batch", title="AUTOZI Latent - Batch Mixing")

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy
import numpy as np

X = latent
labels = test_model.adata.obs['batch'].values

n_neighbors = 30
knn = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
distances, indices = knn.kneighbors(X)

batch_entropies = []
for i in range(X.shape[0]):
    neighbor_batches = labels[indices[i][1:]]  # exclude self
    _, counts = np.unique(neighbor_batches, return_counts=True)
    batch_entropies.append(entropy(counts, base=2))  # base-2 for interpretability

# Compute the true max entropy for a uniform distribution over batches given n_neighbors
n_batches = len(np.unique(labels))
ideal_counts = np.full(n_batches, n_neighbors / n_batches)
max_entropy = entropy(ideal_counts, base=2)

print("Max possible entropy with", n_batches, "batches and", n_neighbors, "neighbors:", max_entropy)
print("Mean entropy of batch mixing:", np.mean(batch_entropies))

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Get denoised expression as a NumPy array
denoised_expr = test_model.get_normalized_expression(library_size=10000, 
                                                                   batch_key = "batch"
                                                                  )  # Normalized per 10,000 reads

# Assign denoised expression as a new layer (ensure format is compatible with AnnData)
test_model.adata.layers["denoised"] = denoised_expr.values  # Convert DataFrame to NumPy array

# Apply log1p transformation
test_model.adata.layers["log_denoised"] = np.log1p(test_model.adata.layers["denoised"])

In [ ]:
# List of gene prefixes
gene_prefixes = ['CD3D:', 'CD3E:', 'CD3G:']

# Get all matching features (e.g., transcript IDs) that start with these prefixes
matching_features = [var for var in test_model.adata.var_names if any(var.startswith(prefix) for prefix in gene_prefixes)]

# Plot UMAP for all matching features (you can adjust how many to show)
sc.pl.umap(test_model.adata, color=matching_features, layer="log_denoised")

In [ ]:
import numpy as np

# Define gene name prefixes
gene_prefixes = ['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654']

# Get all isoforms (var_names) matching those prefixes
matching_features = [var for var in test_model.adata.var_names 
                     if any(var.startswith(prefix) for prefix in gene_prefixes)]

# Extract the log-transformed expression layer
X_layer = test_model.adata[:, matching_features].layers["log_denoised"]

# Sum across isoforms per cell
CD3_sum = X_layer.sum(axis=1)

# Flatten to 1D array
if not isinstance(CD3_sum, np.ndarray):
    CD3_sum = CD3_sum.A1  # for sparse matrices

# Store in obs
test_model.adata.obs['CD3_Combined'] = CD3_sum

# Plot
sc.pl.umap(
    test_model.adata,
    color='CD3_Combined',
    cmap="coolwarm",
    frameon=True,
    title="Total TCell Marker Expression (Summed Log-Denoised Isoforms)",
    vmin=-np.percentile(CD3_sum, 100),
    vmax=np.percentile(CD3_sum, 100),
    show=True
)

In [ ]:
# === Load your MuData ===
# (Assumes you already loaded it like this)
adata = mdata_g_std.mod['rna']

# === Run test ===
test_model = run_autozi_test(
    adata_full=adata,
    subset_n=5000,
    n_latent=40,
    dropout_rate=0.4,
    learning_rate=5e-3,
    weight_decay=1e-2
)

In [ ]:
# Access training history
import matplotlib.pyplot as plt

train_elbo = test_model.history['elbo_train']  # Dataset 1
val_elbo = test_model.history['elbo_validation']  # Dataset 1

# Print last few values
print("Training ELBO gene-level:", train_elbo[-10:])
print("Validation ELBO gene-level:", val_elbo[-10:])

In [ ]:
plt.plot(train_elbo, label="Training ELBO")
plt.plot(val_elbo, label="Validation ELBO")
plt.xlabel("Epoch")
plt.ylabel("Negative ELBO")
plt.legend()
plt.title("Training vs Validation Loss (ELBO) Gene-Level")
plt.show()

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)
sc.pl.umap(test_model.adata, color="batch", title="AUTOZI Latent - Batch Mixing")

from sklearn.metrics import silhouette_score
print(silhouette_score(latent, test_model.adata.obs['batch']))

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy
import numpy as np

X = latent
labels = test_model.adata.obs['batch'].values

n_neighbors = 30
knn = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
distances, indices = knn.kneighbors(X)

batch_entropies = []
for i in range(X.shape[0]):
    neighbor_batches = labels[indices[i][1:]]  # exclude self
    _, counts = np.unique(neighbor_batches, return_counts=True)
    batch_entropies.append(entropy(counts))

print("Mean entropy of batch mixing:", np.mean(batch_entropies))

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Get denoised expression as a NumPy array
denoised_expr = test_model.get_normalized_expression(library_size=10000, 
                                                                   #batch_key = "batch"
                                                                  )  # Normalized per 10,000 reads

# Assign denoised expression as a new layer (ensure format is compatible with AnnData)
test_model.adata.layers["denoised"] = denoised_expr.values  # Convert DataFrame to NumPy array

# Apply log1p transformation
test_model.adata.layers["log_denoised"] = np.log1p(test_model.adata.layers["denoised"])

In [ ]:
sc.pl.umap(test_model.adata, color=['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654'], layer = "log_denoised")

In [ ]:
# Genes of interest
genes = ['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654']

# Calculate summed expression per cell
test_model.adata.obs['CD3_Combined'] = test_model.adata[:, genes].X.toarray().sum(axis=1)

# Plot UMAP
fig = sc.pl.umap(
    test_model.adata,
    color='CD3_Combined',
    cmap= "coolwarm",
    frameon=True,
    title="Total TCell Marker Expression",
    layer = "log_denoised",
    vmin=(-(np.percentile(test_model.adata.obs['CD3_Combined'], 100))),
    vmax=np.percentile(test_model.adata.obs['CD3_Combined'], 100),
    show = True
)

In [ ]:
# === Load your MuData ===
# (Assumes you already loaded it like this)
adata = mdata_g_std.mod['rna']

# === Run test ===
test_model = run_autozi_test(
    adata_full=adata,
    subset_n=5000,
    n_latent=50,
    dropout_rate=0.4,
    learning_rate=5e-3,
    weight_decay=1e-2
)

In [ ]:
# Access training history
import matplotlib.pyplot as plt

train_elbo = test_model.history['elbo_train']  # Dataset 1
val_elbo = test_model.history['elbo_validation']  # Dataset 1

# Print last few values
print("Training ELBO gene-level:", train_elbo[-10:])
print("Validation ELBO gene-level:", val_elbo[-10:])

In [ ]:
plt.plot(train_elbo, label="Training ELBO")
plt.plot(val_elbo, label="Validation ELBO")
plt.xlabel("Epoch")
plt.ylabel("Negative ELBO")
plt.legend()
plt.title("Training vs Validation Loss (ELBO) Gene-Level")
plt.show()

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)
sc.pl.umap(test_model.adata, color="batch", title="AUTOZI Latent - Batch Mixing")

from sklearn.metrics import silhouette_score
print(silhouette_score(latent, test_model.adata.obs['batch']))

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy
import numpy as np

X = latent
labels = test_model.adata.obs['batch'].values

n_neighbors = 30
knn = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
distances, indices = knn.kneighbors(X)

batch_entropies = []
for i in range(X.shape[0]):
    neighbor_batches = labels[indices[i][1:]]  # exclude self
    _, counts = np.unique(neighbor_batches, return_counts=True)
    batch_entropies.append(entropy(counts))

print("Mean entropy of batch mixing:", np.mean(batch_entropies))

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Get denoised expression as a NumPy array
denoised_expr = test_model.get_normalized_expression(library_size=10000, 
                                                                   #batch_key = "batch"
                                                                  )  # Normalized per 10,000 reads

# Assign denoised expression as a new layer (ensure format is compatible with AnnData)
test_model.adata.layers["denoised"] = denoised_expr.values  # Convert DataFrame to NumPy array

# Apply log1p transformation
test_model.adata.layers["log_denoised"] = np.log1p(test_model.adata.layers["denoised"])

In [ ]:
# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)

In [ ]:
sc.pl.umap(test_model.adata, color=['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654'], layer = "log_denoised")

In [ ]:
# Genes of interest
genes = ['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654']

# Calculate summed expression per cell
test_model.adata.obs['CD3_Combined'] = test_model.adata[:, genes].X.toarray().sum(axis=1)

# Plot UMAP
fig = sc.pl.umap(
    test_model.adata,
    color='CD3_Combined',
    cmap= "coolwarm",
    frameon=True,
    title="Total TCell Marker Expression",
    layer = "log_denoised",
    vmin=(-(np.percentile(test_model.adata.obs['CD3_Combined'], 100))),
    vmax=np.percentile(test_model.adata.obs['CD3_Combined'], 100),
    show = True
)

In [ ]:
# === Load your MuData ===
# (Assumes you already loaded it like this)
adata = mdata_g_std.mod['rna']

# === Run test ===
test_model = run_autozi_test(
    adata_full=adata,
    subset_n=5000,
    n_latent=40,
    dropout_rate=0.5,
    learning_rate=5e-3,
    weight_decay=1e-2
)

In [ ]:
# Access training history
import matplotlib.pyplot as plt

train_elbo = test_model.history['elbo_train']  # Dataset 1
val_elbo = test_model.history['elbo_validation']  # Dataset 1

# Print last few values
print("Training ELBO gene-level:", train_elbo[-10:])
print("Validation ELBO gene-level:", val_elbo[-10:])

In [ ]:
plt.plot(train_elbo, label="Training ELBO")
plt.plot(val_elbo, label="Validation ELBO")
plt.xlabel("Epoch")
plt.ylabel("Negative ELBO")
plt.legend()
plt.title("Training vs Validation Loss (ELBO) Gene-Level")
plt.show()

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)
sc.pl.umap(test_model.adata, color="batch", title="AUTOZI Latent - Batch Mixing")

from sklearn.metrics import silhouette_score
print(silhouette_score(latent, test_model.adata.obs['batch']))

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy
import numpy as np

X = latent
labels = test_model.adata.obs['batch'].values

n_neighbors = 30
knn = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
distances, indices = knn.kneighbors(X)

batch_entropies = []
for i in range(X.shape[0]):
    neighbor_batches = labels[indices[i][1:]]  # exclude self
    _, counts = np.unique(neighbor_batches, return_counts=True)
    batch_entropies.append(entropy(counts))

print("Mean entropy of batch mixing:", np.mean(batch_entropies))

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Get denoised expression as a NumPy array
denoised_expr = test_model.get_normalized_expression(library_size=10000, 
                                                                   #batch_key = "batch"
                                                                  )  # Normalized per 10,000 reads

# Assign denoised expression as a new layer (ensure format is compatible with AnnData)
test_model.adata.layers["denoised"] = denoised_expr.values  # Convert DataFrame to NumPy array

# Apply log1p transformation
test_model.adata.layers["log_denoised"] = np.log1p(test_model.adata.layers["denoised"])

In [ ]:
# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)

In [ ]:
sc.pl.umap(test_model.adata, color=['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654'], layer = "log_denoised")

In [ ]:
# Genes of interest
genes = ['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654']

# Calculate summed expression per cell
test_model.adata.obs['CD3_Combined'] = test_model.adata[:, genes].X.toarray().sum(axis=1)

# Plot UMAP
fig = sc.pl.umap(
    test_model.adata,
    color='CD3_Combined',
    cmap= "coolwarm",
    frameon=True,
    title="Total TCell Marker Expression",
    layer = "log_denoised",
    vmin=(-(np.percentile(test_model.adata.obs['CD3_Combined'], 100))),
    vmax=np.percentile(test_model.adata.obs['CD3_Combined'], 100),
    show = True
)

In [ ]:
# === Load your MuData ===
# (Assumes you already loaded it like this)
adata = mdata_g_std.mod['rna']

# === Run test ===
test_model = run_autozi_test(
    adata_full=adata,
    subset_n=5000,
    n_latent=40,
    dropout_rate=0.3,
    learning_rate=5e-3,
    weight_decay=1e-2
)

In [ ]:
# Access training history
import matplotlib.pyplot as plt

train_elbo = test_model.history['elbo_train']  # Dataset 1
val_elbo = test_model.history['elbo_validation']  # Dataset 1

# Print last few values
print("Training ELBO gene-level:", train_elbo[-10:])
print("Validation ELBO gene-level:", val_elbo[-10:])

In [ ]:
plt.plot(train_elbo, label="Training ELBO")
plt.plot(val_elbo, label="Validation ELBO")
plt.xlabel("Epoch")
plt.ylabel("Negative ELBO")
plt.legend()
plt.title("Training vs Validation Loss (ELBO) Gene-Level")
plt.show()

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)
sc.pl.umap(test_model.adata, color="batch", title="AUTOZI Latent - Batch Mixing")

from sklearn.metrics import silhouette_score
print(silhouette_score(latent, test_model.adata.obs['batch']))

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy
import numpy as np

X = latent
labels = test_model.adata.obs['batch'].values

n_neighbors = 30
knn = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
distances, indices = knn.kneighbors(X)

batch_entropies = []
for i in range(X.shape[0]):
    neighbor_batches = labels[indices[i][1:]]  # exclude self
    _, counts = np.unique(neighbor_batches, return_counts=True)
    batch_entropies.append(entropy(counts))

print("Mean entropy of batch mixing:", np.mean(batch_entropies))

In [ ]:
# Use the same adata subset used to train `test_model`
latent = test_model.get_latent_representation(test_model.adata)

# Assign to the same subset
test_model.adata.obsm["X_AUTOZI_test"] = latent

# Get denoised expression as a NumPy array
denoised_expr = test_model.get_normalized_expression(library_size=10000, 
                                                                   #batch_key = "batch"
                                                                  )  # Normalized per 10,000 reads

# Assign denoised expression as a new layer (ensure format is compatible with AnnData)
test_model.adata.layers["denoised"] = denoised_expr.values  # Convert DataFrame to NumPy array

# Apply log1p transformation
test_model.adata.layers["log_denoised"] = np.log1p(test_model.adata.layers["denoised"])

In [ ]:
# Then UMAP
sc.pp.neighbors(test_model.adata, use_rep="X_AUTOZI_test")
sc.tl.umap(test_model.adata)

In [ ]:
sc.pl.umap(test_model.adata, color=['CD3D:ENSG00000167286', 'CD3E:ENSG00000198851', 'CD3G:ENSG00000160654'], layer = "log_denoised")